In [2]:
 from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
 from sklearn.ensemble import RandomForestClassifier
 from sklearn.model_selection import train_test_split
 from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
 import pandas as pd
 import mlflow.sklearn
 import seaborn as sns
 import matplotlib.pyplot as plt
 import os

In [3]:
 from imblearn.over_sampling import SMOTE, ADASYN
 from imblearn.under_sampling   import RandomUnderSampler
 from imblearn.pipeline import Pipeline
 from imblearn.combine import SMOTEENN # Import SMOTEENN



In [4]:
df = pd.read_csv(r"C:\Users\Deepu\OneDrive\Desktop\Capstone1\reddit_preprocessing.csv").dropna(subset=['clean_comment'])

In [ ]:
import mlflow
mlflow.set_tracking_uri("")


In [ ]:
# Step 1: Function to run the experiment
def run_imbalanced_experiment(imbalance_method):
    ngram_range = (1, 3)  # Trigram setting
    max_features = 1000   # Set max_features to 1000 for TF-IDF

    # Step 4: Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        df['clean_comment'], df['category'], 
        test_size=0.2, random_state=42, stratify=df['category']  # FIXED y -> df['category']
    )

    # Step 2: Vectorization using TF-IDF
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 3: Handle class imbalance based on the selected method
    if imbalance_method == 'class_weights':
        class_weight = 'balanced'
    else:
        class_weight = None  # Do not apply class_weight if using resampling

        # Resampling Techniques (FIXED X, y -> X_train, y_train)
        if imbalance_method == 'oversampling':
            smote = SMOTE(random_state=42)
            X_train, y_train = smote.fit_resample(X_train, y_train)
        elif imbalance_method == 'adasyn':
            adasyn = ADASYN(random_state=42)
            X_train, y_train = adasyn.fit_resample(X_train, y_train)
        elif imbalance_method == 'undersampling':
            rus = RandomUnderSampler(random_state=42)
            X_train, y_train = rus.fit_resample(X_train, y_train)
        elif imbalance_method == 'smote_enn':
            smote_enn = SMOTEENN(random_state=42)
            X_train, y_train = smote_enn.fit_resample(X_train, y_train)

    # Step 5: Define and train a Random Forest model
    with mlflow.start_run() as run:
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")
        mlflow.set_tag(
            "description",
            f"RandomForest with TF-IDF Trigrams, imbalance handling method={imbalance_method}"
        )

        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        model = RandomForestClassifier(class_weight=class_weight, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path=f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}",
            registered_model_name="RandomForest_TFIDF_Trigrams"
        )

# Step 7: Run experiments for different imbalance methods
imbalance_methods = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']
for method in imbalance_methods:
    run_imbalanced_experiment(method)
